## ⚠️ POPPLER & TESSERACT REQUIRED FOR OCR

**These PDFs are scanned images** - pdfplumber can only extract headers. OCR is required to read the actual content.

### Install Poppler (Windows):
1. Download: https://github.com/oschwartz10612/poppler-windows/releases/latest
2. Extract to `C:\poppler` (or any location)
3. **Option A**: Add `C:\poppler\Library\bin` to your Windows PATH
4. **Option B**: Or specify the path in code (see POPPLER_PATH variable below)

### Install Tesseract OCR (Windows):
1. Download installer: https://github.com/UB-Mannheim/tesseract/wiki
2. Run the installer (default location: `C:\Program Files\Tesseract-OCR`)
3. Add `C:\Program Files\Tesseract-OCR` to your Windows PATH
4. Or set `pytesseract.pytesseract.tesseract_cmd` in code

After installation, restart this notebook kernel and re-run the cells.

# CRC UDS Prototype (2025)
This notebook ingests:
- Scanned CRC reports (PDF) via OCR
- FOBT CSV extract

It outputs per-patient CRC numerator status for 2025 with an auditor-friendly evidence string.


In [32]:
import os, re, json, datetime as dt
import pandas as pd
from dateutil.relativedelta import relativedelta
import pytesseract
from pdf2image import convert_from_path


In [33]:
import glob, os

# Auto-discover CRC-related PDFs in /mnt/data
PDF_GLOBS = [
    "crc_pdf/*colonoscopy*.pdf",
    "crc_pdf/*_fit*.pdf",
    "crc_pdf/*_ifobt*.pdf",
    "crc_pdf/*fecal_ia*.pdf",
    "crc_pdf/*fobt*.pdf",
]
PDF_PATHS = sorted({p for g in PDF_GLOBS for p in glob.glob(g)})

FOBT_CSV_PATH = "crc_csv_data/fobt.csv"
REPORTING_YEAR = 2025

print(f"Found {len(PDF_PATHS)} PDFs:")
for p in PDF_PATHS:
    print(" -", os.path.basename(p))
print("FOBT CSV:", os.path.basename(FOBT_CSV_PATH), "exists:", os.path.exists(FOBT_CSV_PATH))

Found 16 PDFs:
 - 1079665_colonoscopy.pdf
 - 110656_colonoscopy.pdf
 - 118450_colonoscopy.pdf
 - 12969_colonoscopy.pdf
 - 12969_fit.pdf
 - 151177_ifobt.pdf
 - 165505_fit.pdf
 - 166003_ifobt.pdf
 - 166222_colonoscopy.pdf
 - 166372_colonoscopy.pdf
 - 183636_colonoscopy.pdf
 - 184224_colonoscopy.pdf
 - 190147_ifobt.pdf
 - 46874_fit.pdf
 - 75139_colonoscopy.pdf
 - 9118_colonoscopy.pdf
FOBT CSV: fobt.csv exists: True


In [51]:
import os, re, datetime as dt
from dataclasses import dataclass
from typing import Optional, Literal, Dict, Any, List, Tuple

DATE_PAT = re.compile(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})')
# Match dates with month names like "April 02, 2018" or "April 2, 2018"
MONTH_NAME_PAT = re.compile(r'\b(January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)\.?\s+(\d{1,2}),?\s+(\d{4})\b', re.IGNORECASE)

def _to_iso(m:int,d:int,y:int) -> Optional[str]:
    if y < 100:
        y = 2000 + y if y < 50 else 1900 + y
    try:
        return dt.date(y,m,d).isoformat()
    except Exception:
        return None

def parse_dates(text: str) -> List[Tuple[str, str]]:
    out=[]
    # Extract numeric dates like 4/2/2018
    for m,d,y in DATE_PAT.findall(text or ""):
        iso = _to_iso(int(m), int(d), int(y))
        if iso:
            out.append((iso, f"{m}/{d}/{y}"))
    
    # Extract dates with month names like "April 02, 2018"
    month_map = {
        'jan': 1, 'january': 1, 'feb': 2, 'february': 2, 'mar': 3, 'march': 3,
        'apr': 4, 'april': 4, 'may': 5, 'jun': 6, 'june': 6,
        'jul': 7, 'july': 7, 'aug': 8, 'august': 8, 'sep': 9, 'sept': 9, 'september': 9,
        'oct': 10, 'october': 10, 'nov': 11, 'november': 11, 'dec': 12, 'december': 12
    }
    for month_name, day, year in MONTH_NAME_PAT.findall(text or ""):
        month_num = month_map.get(month_name.lower())
        if month_num:
            iso = _to_iso(month_num, int(day), int(year))
            if iso:
                out.append((iso, f"{month_name} {day}, {year}"))
    
    return out

def looks_like_nonclinical_line(line: str) -> bool:
    l=(line or "").lower()
    if "electronically signed" in l or "signed by" in l:
        return True
    if "dob" in l or "date of birth" in l:
        return True
    if "data:text" in l or "base64" in l:
        return True
    # Filter out fax cover sheet lines
    if "from: fax" in l or ("from:" in l and "to:" in l and "page:" in l):
        return True
    # Filter out historical references to previous procedures
    if "last colonoscopy" in l or "previous colonoscopy" in l or "prior colonoscopy" in l:
        return True
    if "last procedure" in l or "previous procedure" in l:
        return True
    # Filter out future recommendations and follow-up instructions
    if "recommend" in l and ("colonoscopy" in l or "repeat" in l or "follow" in l):
        return True
    if "next colonoscopy" in l or "follow-up colonoscopy" in l:
        return True
    return False

def filename_patient_id(path: str) -> str:
    base=os.path.basename(path)
    m=re.match(r'^(\d+)', base)
    return m.group(1) if m else base

import pdfplumber
from pdf2image import convert_from_path
import pytesseract

KEYWORDS = ["colonoscopy","occult","fecal","fit","fobt","ifobt","colofit","guaiac"]

# POPPLER PATH: Uncomment and set this if poppler is not in your system PATH
POPPLER_PATH = r"C:\poppler\Library\bin"

# TESSERACT PATH: Set path to tesseract executable
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\jloya\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
#POPPLER_PATH = None  # Set to None to use system PATH

def extract_first_page_text(pdf_path: str, ocr_dpi: int = 300, min_chars: int = 80, max_pages: int = 5) -> str:
    """Extract text from PDF, checking multiple pages if needed. Uses OCR fallback for scanned documents."""
    all_text = ""
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            pages_to_check = min(max_pages, total_pages)
            
            # Try extracting from first few pages
            for page_num in range(pages_to_check):
                page_text = (pdf.pages[page_num].extract_text() or "")
                all_text += page_text + "\n"
                
                # Check if we have enough clinical content AND dates
                text_l = all_text.lower()
                keyword_hit = any(k in text_l for k in KEYWORDS)
                import re
                has_dates = bool(re.search(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', all_text))
                
                # Only stop if we have keywords AND dates (not just DOB)
                if len(all_text.strip()) >= min_chars and keyword_hit and has_dates:
                    # Check it's not just DOB
                    date_lines = [ln for ln in all_text.split('\n') if re.search(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', ln)]
                    non_dob_dates = [ln for ln in date_lines if 'dob' not in ln.lower() and 'date of birth' not in ln.lower()]
                    if non_dob_dates:
                        return all_text
    except Exception:
        pass

    # If we didn't get good text with dates, try OCR on first few pages
    if len(all_text.strip()) < min_chars or not any(k in all_text.lower() for k in KEYWORDS):
        try:
            ocr_combined = ""
            # OCR up to max_pages or until we find clinical content WITH dates
            for page_num in range(1, min(max_pages + 1, 15)):  # Cap at 15 pages max
                pages = convert_from_path(
                    pdf_path, 
                    dpi=ocr_dpi, 
                    first_page=page_num, 
                    last_page=page_num,
                    poppler_path=POPPLER_PATH
                )
                if pages:
                    ocr_text = pytesseract.image_to_string(pages[0], timeout=12)
                    ocr_combined += ocr_text + "\n"
                    
                    # Check if we found clinical content WITH dates
                    ocr_l = ocr_combined.lower()
                    keyword_hit = any(k in ocr_l for k in KEYWORDS)
                    import re
                    has_dates = bool(re.search(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', ocr_combined))
                    
                    if len(ocr_combined.strip()) >= min_chars and keyword_hit and has_dates:
                        # Check it's not just DOB
                        date_lines = [ln for ln in ocr_combined.split('\n') if re.search(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', ln)]
                        non_dob_dates = [ln for ln in date_lines if 'dob' not in ln.lower() and 'date of birth' not in ln.lower()]
                        if non_dob_dates:
                            return ocr_combined
                        
            # Return whatever OCR we got if it's better than pdfplumber
            if len(ocr_combined.strip()) > len(all_text.strip()):
                return ocr_combined
        except Exception as e:
            pass
    
    # Return whatever we got, even if minimal
    return all_text or ""

EventType = Literal["FOBT","FIT","FIT_DNA","SIGMOIDOSCOPY","CT_COLONOGRAPHY","COLONOSCOPY"]

@dataclass
class ScreeningEvent:
    patient_id: str
    event_type: EventType
    event_date: Optional[str]
    source: Literal["pdf","structured_csv"]
    confidence: float
    needs_review: bool
    evidence: Dict[str, Any]

def classify_doc(text: str) -> Optional[EventType]:
    t=(text or "").lower()
    if "colonoscopy" in t:
        return "COLONOSCOPY"
    if "occult blood" in t and "fecal" in t:
        return "FIT"  # includes IA (FIT)
    if "colofit" in t:
        return "FIT"
    if "\bfit\b" in t and "fecal" in t:
        return "FIT"
    if "ifobt" in t or "fobt" in t or "guaiac" in t:
        return "FOBT"
    return None

def extract_labeled_dates(text: str) -> Dict[str, List[str]]:
    labels = {"procedure":[],"collection":[],"received":[],"result":[],"other":[]}
    lines = (text or "").splitlines()
    
    # First pass: identify lines with strong procedure date indicators but potentially garbled dates
    procedure_date_line_indices = []
    for i, line in enumerate(lines):
        if not line.strip() or looks_like_nonclinical_line(line):
            continue
        ll = line.lower()
        # Strong indicators that this section contains the procedure date
        if re.search(r'\bprocedure\b.*\bdate\b', ll) or re.search(r'\bdate\s*of\s*operation\b', ll):
            procedure_date_line_indices.append(i)
    
    # Second pass: label dates based on context
    for i, line in enumerate(lines):
        if not line.strip():
            continue
        if looks_like_nonclinical_line(line):
            continue
        ll=line.lower()
        label="other"
        
        # Check if we're near a procedure date indicator line (within 2 lines)
        near_procedure_indicator = any(abs(i - proc_idx) <= 2 for proc_idx in procedure_date_line_indices)
        
        # For colonoscopy reports, "Date:" near MRN or Patient info is the procedure date
        if re.search(r'\bprocedure\b.*\bdate\b', ll) or re.search(r'procedure\s*date|date\s*of\s*procedure|date\s*of\s*service|\bdos\b', ll):
            label="procedure"
        # Match "date of operation" or "operation date"
        elif re.search(r'\bdate\s*of\s*operation\b|\boperation\s*date\b', ll):
            label="procedure"
        # If we're near a procedure date indicator and this line has a date, label it as procedure
        elif near_procedure_indicator and re.search(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', ll):
            label="procedure"
        # Match "performed on" - common in narrative reports (may have month name dates)
        elif re.search(r'\bperformed\b.*(on|at)', ll):
            label="procedure"
        # Match lines with "MRN" and "Date:" - common in procedure reports
        elif re.search(r'\bmrn\b.*\bdate\s*:', ll):
            label="procedure"
        # Handle NUL bytes in "Collection Date" from font encoding issues
        elif re.search(r'\bcollec[\s\x00]*[ti]?on\b.*\bdate\b', ll) or re.search(r'\bcollect\b.*\bdate\b', ll):
            label="collection"
        elif re.search(r'collection\s*date|date\s*of\s*collection', ll):
            label="collection"
        # Match "Test date" - common in lab reports for collection/specimen date
        elif re.search(r'\btest\s*date\b', ll):
            label="collection"
        # Match "returned as" or "result...on DATE" - common in FIT results letters
        elif (re.search(r'\breturned\b.*(as|on)', ll) or re.search(r'\bresult.*\b(as|on)\b', ll)) and re.search(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', ll):
            label="collection"
        # Match "Order Date" - typically the date procedure was ordered/collected
        elif re.search(r'\border\b.*\bdate\b', ll):
            label="collection"
        # Match "obtained:" or "collected on/at"
        elif re.search(r'\b(collected|obtained)\b.*(on|at|:)', ll):
            label="collection"
        # Match "Final, MM/DD/YYYY" pattern
        elif re.search(r'\bfinal\s*,\s*\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', ll):
            label="collection"
        elif re.search(r'\breceived\b', ll):
            label="received"
        # Match "Report:" patterns
        elif re.search(r'\breport(ed)?\s*:', ll):
            label="result"
        # Don't match "finalized" as result - too generic
        elif re.search(r'\bresult\b|\breported\b', ll):
            label="result"
        # REMOVED: generic "final" pattern that was catching "Finalized Date"
        # Override with "at/on MM/DD/YYYY" pattern for procedure dates
        if re.search(r'\b(at|on)\s+\w+,?\s*\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b', ll):
            label="procedure"
        for iso,_ in parse_dates(line):
            labels[label].append(iso)
    for k,v in labels.items():
        seen=set(); nv=[]
        for d in v:
            if d not in seen:
                nv.append(d); seen.add(d)
        labels[k]=nv
    return labels

def is_reasonable_procedure_date(date_iso: str) -> bool:
    """Filter out dates that are clearly not procedure dates (e.g., DOBs from 1950s, future dates)"""
    try:
        year = int(date_iso[:4])
        current_year = dt.date.today().year
        # CRC screening procedures should be in a reasonable range
        # Allow 1995 to current year (no future dates - likely OCR errors)
        return 1995 <= year <= current_year
    except:
        return False

def choose_best_date(event_type: EventType, labeled: Dict[str,List[str]], all_dates: List[str]) -> Tuple[Optional[str], float, bool, str]:
    if event_type=="COLONOSCOPY":
        pref="procedure"
    elif event_type in ["FIT","FOBT"]:
        pref="collection"
    else:
        pref="other"
    
    # Filter out invalid dates (DOBs, future dates) from all categories first
    for key in labeled:
        labeled[key] = [d for d in labeled[key] if is_reasonable_procedure_date(d)]

    candidates = labeled.get(pref, []).copy()
    if not candidates:
        # For fallback, prefer result (report date) over received date for accuracy
        candidates = labeled.get("result",[]) + labeled.get("received",[]) + labeled.get("other",[])
    if not candidates and all_dates:
        candidates = [d for d in all_dates if is_reasonable_procedure_date(d)]
        candidates = list(dict.fromkeys(candidates))

    if not candidates:
        return None, 0.2, True, "no date found"

    best = max(candidates)
    conf = 0.9 if (labeled.get(pref) and best in labeled[pref]) else 0.65
    needs_review = False
    rationale = f"picked {best} from {pref if conf>0.8 else 'fallback'}"

    distinct = sorted(set(sum(labeled.values(), [])))
    if conf < 0.8 and len(distinct) >= 2:
        needs_review = True
        conf = min(conf, 0.55)
        rationale += "; multiple conflicting dates"

    return best, conf, needs_review, rationale

def extract_event_from_pdf(pdf_path: str, text: str) -> ScreeningEvent:
    pid = filename_patient_id(pdf_path)
    filename = os.path.basename(pdf_path).lower()
    
    # FIXED: Use filename as hint if text extraction failed
    et = classify_doc(text)
    if et is None:
        # Fallback to filename-based classification
        if "colonoscopy" in filename:
            et = "COLONOSCOPY"
        elif "fit" in filename and "dna" not in filename:
            et = "FIT"
        elif "ifobt" in filename or "fobt" in filename:
            et = "FOBT"
        elif "fecal" in filename:
            et = "FIT"
        else:
            et = "FOBT"  # Last resort default
    
    labeled = extract_labeled_dates(text)
    all_dates = [d for d,_ in parse_dates(text)]
    best_date, conf, needs_review, rationale = choose_best_date(et, labeled, all_dates)

    lines = [ln.strip() for ln in (text or "").splitlines() if ln.strip()]
    if et == "COLONOSCOPY":
        keys = ["procedure", "colonoscopy", "impression", "indication", "findings", "date of service", "dos"]
    else:
        keys = ["collection date", "collected", "occult blood", "fecal", "fit", "fobt", "ifobt", "colofit"]
    snippet_lines = []
    for ln in lines:
        lnl = ln.lower()
        if any(k in lnl for k in keys):
            snippet_lines.append(ln)
        if len(snippet_lines) >= 8:
            break
    snippet = (" ".join(snippet_lines) or (text or "")[:600]).replace(chr(10), " ")

    # If text was minimal (likely failed OCR), mark for review and note filename-based classification
    text_minimal = len(text.strip()) < 100
    if text_minimal:
        needs_review = True
        conf = min(conf, 0.6)
        if best_date:
            rationale += f"; text extraction minimal ({len(text.strip())} chars), classified by filename"
        else:
            rationale = f"text extraction minimal ({len(text.strip())} chars), classified by filename, no valid date"

    # If we found FIT/FOBT but no confident collection/procedure date, flag for review
    if et in ["FIT","FOBT"] and (best_date is None or conf < 0.8):
        needs_review = True

    return ScreeningEvent(
        patient_id=pid,
        event_type=et,
        event_date=best_date,
        source="pdf",
        confidence=float(conf),
        needs_review=bool(needs_review),
        evidence={
            "file": os.path.basename(pdf_path),
            "rationale": rationale,
            "snippet": snippet,
            "dates_by_label": labeled,
            "text_length": len(text),
        }
    )

In [52]:
# Extract events from PDFs (fast text extraction with OCR fallback)
pdf_rows=[]
for pdf_path in PDF_PATHS:
    text = extract_first_page_text(pdf_path)
    ev = extract_event_from_pdf(pdf_path, text)
    pdf_rows.append(ev.__dict__)

df_pdf = pd.DataFrame(pdf_rows)
df_pdf

,patient_id,event_type,event_date,source,confidence,needs_review,evidence
0,1079665,COLONOSCOPY,2023-06-08,pdf,0.90,False,"{'file': '1079665_colonoscopy.pdf', 'rationale..."
1,110656,COLONOSCOPY,2019-12-19,pdf,0.90,False,"{'file': '110656_colonoscopy.pdf', 'rationale'..."
2,118450,COLONOSCOPY,2016-03-30,pdf,0.90,False,"{'file': '118450_colonoscopy.pdf', 'rationale'..."
3,12969,COLONOSCOPY,2012-08-01,pdf,0.90,False,"{'file': '12969_colonoscopy.pdf', 'rationale':..."
4,12969,FIT,2025-03-03,pdf,0.90,False,"{'file': '12969_fit.pdf', 'rationale': 'picked..."
5,151177,FOBT,2025-01-30,pdf,0.90,False,"{'file': '151177_ifobt.pdf', 'rationale': 'pic..."
6,165505,FIT,2023-10-22,pdf,0.65,True,"{'file': '165505_fit.pdf', 'rationale': 'picke..."
7,166003,FIT,2025-04-17,pdf,0.90,False,"{'file': '166003_ifobt.pdf', 'rationale': 'pic..."
8,166222,COLONOSCOPY,2019-03-11,pdf,0.90,False,"{'file': '166222_colonoscopy.pdf', 'rationale'..."
9,166372,COLONOSCOPY,2017-04-06,pdf,0.90,False,"{'file': '166372_colonoscopy.pdf', 'rationale'..."


In [ ]:
# Parse FOBT CSV extract and convert to ScreeningEvent rows
import pandas as pd
import numpy as np
import datetime as dt

def detect_header_row(csv_path: str, max_scan: int = 40) -> int:
    with open(csv_path, 'r', errors='ignore') as f:
        for i,line in enumerate(f):
            if i>max_scan:
                break
            l=line.lower()
            if 'person' in l and ('nbr' in l or '#' in l) and ('mrn' in l or 'enc' in l or 'date' in l):
                return i
    return 0

hdr = detect_header_row(FOBT_CSV_PATH)
df_csv = pd.read_csv(FOBT_CSV_PATH, header=hdr)
df_csv.columns = [c.strip() for c in df_csv.columns]

# Identify key columns
def find_col(cols, patterns):
    for p in patterns:
        for c in cols:
            if re.search(p, c, re.I):
                return c
    return None

pid_col = find_col(df_csv.columns, [r'person\s*nbr', r'person\s*#', r'patient'])
encdate_col = find_col(df_csv.columns, [r'enc\s*date', r'collection\s*date', r'collected', r'date'])
test_col = find_col(df_csv.columns, [r'test', r'order', r'procedure', r'description', r'name'])

def parse_any_date(x):
    if pd.isna(x): return None
    s=str(x).strip()
    # try mm/dd/yyyy
    m=DATE_PAT.search(s)
    if not m: 
        return None
    mm,dd,yy = map(int, m.groups())
    iso=_to_iso(mm,dd,yy)
    return iso

csv_events=[]
for idx,row in df_csv.iterrows():
    pid = str(row.get(pid_col, '')).strip()
    if not pid or pid.lower()=='nan':
        continue
    ev_date = parse_any_date(row.get(encdate_col, None))
    if not ev_date:
        continue
    test_name = str(row.get(test_col, '')).lower() if test_col else ''
    et = 'FIT' if 'fit' in test_name and 'dna' not in test_name else 'FOBT'
    csv_events.append({
        "patient_id": pid,
        "event_type": et,
        "event_date": ev_date,
        "source": "structured_csv",
        "confidence": 0.95,
        "needs_review": False,
        "evidence": {"source_file": os.path.basename(FOBT_CSV_PATH), "row_index": int(idx), "test_name": test_name}
    })

df_csv_events = pd.DataFrame(csv_events)
df_csv_events.head(), df_csv_events.shape

In [ ]:
# CRC 2025 rule engine + merge, de-dup, and scoring
import datetime as dt
from dateutil.relativedelta import relativedelta

year_start = dt.date(REPORTING_YEAR,1,1)
year_end = dt.date(REPORTING_YEAR,12,31)

def counts_for_crc(event_type: str, event_date_iso: str) -> bool:
    if not event_date_iso:
        return False
    d = dt.date.fromisoformat(event_date_iso)
    if event_type in ["FOBT","FIT"]:
        return year_start <= d <= year_end
    if event_type == "FIT_DNA":
        return (year_start - relativedelta(years=2)) <= d <= year_end
    if event_type in ["SIGMOIDOSCOPY","CT_COLONOGRAPHY"]:
        return (year_start - relativedelta(years=4)) <= d <= year_end
    if event_type == "COLONOSCOPY":
        return (year_start - relativedelta(years=9)) <= d <= year_end
    return False

# Merge
df_all = pd.concat([
    df_pdf[["patient_id","event_type","event_date","source","confidence","needs_review","evidence"]],
    df_csv_events[["patient_id","event_type","event_date","source","confidence","needs_review","evidence"]],
], ignore_index=True)

# De-dup: keep highest confidence for same patient/type/date/source
df_all["dedup_key"] = df_all["patient_id"].astype(str)+"|"+df_all["event_type"].astype(str)+"|"+df_all["event_date"].astype(str)
df_all = (df_all
          .sort_values(["confidence","source"], ascending=[False, True])
          .drop_duplicates("dedup_key", keep="first")
          .drop(columns=["dedup_key"])
         )

df_all["counts_crc_2025"] = df_all.apply(lambda r: counts_for_crc(r["event_type"], r["event_date"]), axis=1)
df_all.sort_values(["patient_id","counts_crc_2025","event_date"], ascending=[True, False, False]).head(30)

In [ ]:
# Pick best numerator evidence per patient (most recent qualifying; tie-break by confidence)
def pick_best(df):
    q = df[df["counts_crc_2025"]==True].copy()
    if q.empty:
        # keep best available for review (most recent date regardless) if any
        df2 = df.copy()
        df2 = df2[df2["event_date"].notna()]
        if df2.empty:
            return pd.Series({
                "numerator_met": False,
                "best_event_type": None,
                "best_event_date": None,
                "best_source": None,
                "best_confidence": None,
                "needs_review": True,
                "evidence_summary": "no dated CRC evidence found",
            })
        df2 = df2.sort_values(["event_date","confidence"], ascending=[False,False]).head(1)
        row = df2.iloc[0]
        return pd.Series({
            "numerator_met": False,
            "best_event_type": row.event_type,
            "best_event_date": row.event_date,
            "best_source": row.source,
            "best_confidence": row.confidence,
            "needs_review": True,
            "evidence_summary": f"Best non-qualifying evidence: {row.event_type} {row.event_date} ({row.source}); {row.evidence.get('file', row.evidence.get('source_file',''))}",
        })
    q = q.sort_values(["event_date","confidence"], ascending=[False,False]).head(1)
    row = q.iloc[0]
    file_or_row = row.evidence.get('file', row.evidence.get('source_file',''))
    snippet = row.evidence.get('snippet','')
    summary = f"{row.event_type} on {row.event_date} via {row.source} ({file_or_row})"
    if snippet:
        summary += f" | snippet: {snippet[:160]}"
    return pd.Series({
        "numerator_met": True,
        "best_event_type": row.event_type,
        "best_event_date": row.event_date,
        "best_source": row.source,
        "best_confidence": row.confidence,
        "needs_review": bool(row.needs_review),
        "evidence_summary": summary,
    })

df_best = df_all.groupby("patient_id", as_index=False).apply(pick_best).reset_index(drop=True)
df_best = df_best.sort_values(["numerator_met","needs_review","best_event_date"], ascending=[False, True, False])

# Save auditor table
os.makedirs("audit_data", exist_ok=True)
out_csv = "audit_data/crc_2025_audit_table_updated.csv"
df_best.to_csv(out_csv, index=False, escapechar='\\', doublequote=False)
print(f"✓ Saved audit table to: {out_csv}")
out_csv, df_best.head(50)

In [ ]:
# DEBUG: Check what pdfplumber extracts vs what's actually in the PDF
test_pdf = "pdf_data/110656_colonoscopy.pdf"

print("="*80)
print("PDFPLUMBER EXTRACTION:")
print("="*80)
try:
    with pdfplumber.open(test_pdf) as pdf:
        print(f"Total pages: {len(pdf.pages)}")
        for i, page in enumerate(pdf.pages[:3]):  # Check first 3 pages
            text = page.extract_text()
            print(f"\n--- Page {i+1} (length: {len(text) if text else 0}) ---")
            print(text[:1000] if text else "No text extracted")
            print()
except Exception as e:
    print(f"Error: {e}")

print("\n" + "="*80)
print(f"TRYING OCR with POPPLER_PATH={POPPLER_PATH}:")
print("="*80)
try:
    pages = convert_from_path(test_pdf, dpi=150, first_page=1, last_page=1, poppler_path=POPPLER_PATH)
    if pages:
        ocr_text = pytesseract.image_to_string(pages[0])
        print(f"✓ OCR SUCCESS! Extracted {len(ocr_text)} chars")
        print(f"OCR text (first 1000 chars):\n{ocr_text[:1000]}")
except Exception as e:
    print(f"✗ OCR failed: {e}")